## 2 Text Cleaning for training data

In [26]:
#text cleaning 
#reference:
#https://machinelearningmastery.com/clean-text-machine-learning-python/

In [1]:
# #import clean training and hugging face chat LLM created training data
import pandas as pd
#ingest original training data with LLM created training data 
OG_df = pd.read_csv(r'Data/original_training_LLMcreated.csv')
OG_df
#df.id.value_counts()

,id,class,text
0,report_519.pdf.53,governance,2021 ESG Impact Report ...
1,report_1537.pdf.24,social,24 2021 SUSTAINABILITY REPORT TABLE OF CONTENT...
2,report_567.pdf.33,governance,Sustainability Governance Clean Harbors’ commi...
3,report_1830.pdf.220,other,220 Report of the réviseur d’entreprises agréé...
4,report_1253.pdf.46,governance,Pfizer 2021 ESG Report 46 Governance Governanc...
...,...,...,...
2409,LLM created,governance,Everyone in the organization has a role to pla...
2410,LLM created,governance,We aim to create a workplace where people feel...
2411,LLM created,governance,Our approach to corporate citizenship means gi...
2412,LLM created,governance,By building an open and inclusive corporate cu...


In [2]:
print(len(OG_df))

2414


In [11]:
#import the training txt files, parse and append to a new df
import pandas as pd 
# Get Hugging Face LLM created texts
my_file = open('Data/oxml_esg_texts.txt', "r")
data = my_file.read()
texts = data.split("unique_linebreak")
my_file.close()

# Get ESG labels
df = pd.read_csv('Data/labels.csv')
print("length of OG train")
print(len(df))

labels = {
    'governance': 0,
    'social': 1,
    'environmental': 2,
    'other': 3
}

# Add text column to label dataframe
df['text'] = texts[:-1]

# ''' -------------------------------------------------------------------- '''



# ''' -------------------------------------------------------------------- '''
#start with empty dataframe
import pandas as pd 
#df = pd.DataFrame()

# # Get extra environmental texts
my_file = open('Data/environment_trianingGPT4.txt', "r")
data = my_file.read()
env_texts = data.split("unique_linebreak")
my_file.close()

# Create environmental labels for extra texts
env_labels = ['environmental'] * len(env_texts)

# Add extra environmental texts to working dataframe
for i, j in zip(env_texts, env_labels):
    df = df.append({'text': i, 'class': j}, ignore_index=True)

''' -------------------------------------------------------------------- '''

# Get extra social texts
my_file = open('Data/social_trianingGPT3.txt', "r")
data = my_file.read()
social_texts = data.split("unique_linebreak")
my_file.close()

# Create social labels for extra texts
social_labels = ['social'] * len(social_texts)

# Add extra social texts to working dataframe
for i, j in zip(social_texts, social_labels):
    df = df.append({'text': i, 'class': j}, ignore_index=True)

''' -------------------------------------------------------------------- '''


# # Get extra governance texts
my_file = open('Data/governance_trianingGPT3.txt', "r")
data = my_file.read()
gov_texts = data.split("unique_linebreak")
my_file.close()

# Create social labels for extra texts
gov_labels = ['governance'] * len(gov_texts)

# Add extra social texts to working dataframe
for i, j in zip(gov_texts, gov_labels):
    df = df.append({'text': i, 'class': j}, ignore_index=True)
print("new length")
print(len(df))

length of OG train
1956
new length
2117


In [12]:
#create ID field 
df['id'] = 'GPT4All'
#rearrange columns
df = df[['id', 'class', 'text']]


In [13]:
#append OG + LLM to new GPT4All data
df_appended = OG_df.append(df)
print(len(df_appended))

4531


In [ ]:
#text cleaning 

In [14]:
#remove duplicate text values 
df_appended = df_appended.drop_duplicates(subset=['text'], keep='first')
print(len(df_appended))

4439


In [15]:
#remove punctuation and \n 
import re 
df = df_appended 
#df['text'] = df['text'].str.replace('[^\w\s]','')


# Match all digits in the string and replace them with an empty string
df['text'] = df['text'].astype(str)
df['text'] = df['text'].str.replace('\d+', '')

df['text'] = df['text'].str.replace('\n', '')

#lower
#df['text'] = df['text'].str.lower()
df['text']

0        ESG Impact Report                            ...
1         SUSTAINABILITY REPORT TABLE OF CONTENTS SOCI...
2       Sustainability Governance Clean Harbors’ commi...
3        Report of the réviseur d’entreprises agréé • ...
4       Pfizer  ESG Report  Governance Governance — Co...
                              ...                        
2111    In this report, we will analyze GPT's commitme...
2112    In order to ensure successful implementation o...
2113    I am pleased to provide you with this report a...
2114    As an AI language model, I can generate  words...
2115    I am pleased to present this report on XYZ Com...
Name: text, Length: 4439, dtype: object

In [16]:
#Filter Out non alpha numeric
import re
import nltk
from nltk.tokenize import word_tokenize
# remove all tokens that are not alphabetic
df['text'] = df['text'].astype(str)

# Remove characters that are not letter or numbers
# Define a function to remove non-alphanumeric tokens
def remove_non_alphanumeric(text):
    tokens = word_tokenize(text)
    alphanumeric_tokens = [token for token in tokens if token.isalnum()]
    return ' '.join(alphanumeric_tokens)

# Apply the function to the 'text' column
df['text'] = df['text'].apply(remove_non_alphanumeric)
df['text']


0       ESG Impact Report Data privacy security and re...
1       SUSTAINABILITY REPORT TABLE OF CONTENTS SOCIAL...
2       Sustainability Governance Clean Harbors commit...
3       Report of the réviseur d entreprises agréé Eva...
4       Pfizer ESG Report Governance Governance Contin...
                              ...                        
2111    In this report we will analyze GPT commitment ...
2112    In order to ensure successful implementation o...
2113    I am pleased to provide you with this report a...
2114    As an AI language model I can generate words o...
2115    I am pleased to present this report on XYZ Com...
Name: text, Length: 4439, dtype: object

In [9]:
# # Initialize the PorterStemmer
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize

# stemmer = PorterStemmer()

# # Define a function to perform stemming
# def stem_words(text):
#     tokens = word_tokenize(text)
#     stemmed_tokens = [stemmer.stem(token) for token in tokens]
#     return ' '.join(stemmed_tokens)

# # Apply the function to the 'text' column
# df['text'].apply(stem_words)

In [46]:
# #Removing Contractions
# #!pip install contractions
# import contractions
# df['text'].apply(lambda x: [contractions.fix(word) for word in x.split()])


# #Output
# #she would like to know how I would do that!

In [17]:
#upgrade nltk for lemmonization 
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /Users/Jupiter/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [18]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to perform lemmatization
def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Apply the function to the 'text' column
df['text'] = df['text'].apply(lemmatize_text)

In [19]:
#remove stopwords
#remove stop words from content
from nltk.corpus import stopwords

final_stopwords_list = stopwords.words('english')
# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (final_stopwords_list)]))
df['text'] 

0       ESG Impact Report Data privacy security reliab...
1       SUSTAINABILITY REPORT TABLE OF CONTENTS SOCIAL...
2       Sustainability Governance Clean Harbors commit...
3       Report réviseur entreprises agréé Evaluate app...
4       Pfizer ESG Report Governance Governance Contin...
                              ...                        
2111    In report analyze GPT commitment Environmental...
2112    In order ensure successful implementation ESG ...
2113    I pleased provide report Company X focus corpo...
2114    As AI language model I generate word text base...
2115    I pleased present report XYZ Company focus cor...
Name: text, Length: 4439, dtype: object

In [21]:
df.tail()

,id,class,text
2111,GPT4All,governance,In report analyze GPT commitment Environmental...
2112,GPT4All,governance,In order ensure successful implementation ESG ...
2113,GPT4All,governance,I pleased provide report Company X focus corpo...
2114,GPT4All,governance,As AI language model I generate word text base...
2115,GPT4All,governance,I pleased present report XYZ Company focus cor...


In [20]:
#remove empty rows 
#df.dropna(subset=['text'], inplace=True)
df = df.dropna(subset=['text'])
print(len(df))

4439


In [17]:
# #drop last empty row
# df = df[:-1]


In [ ]:
#determine proportion of classes - they are uneven 

In [23]:
df['class'].value_counts(normalize=True)

other            0.400541
social           0.280243
environmental    0.170984
governance       0.148232
Name: class, dtype: float64

In [24]:
#df.to_csv('Data/4463_clean_training.csv')

In [25]:
#imbalance to balance class 
#https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/
#!pip install imbalanced-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 2.5 MB/s eta 0:00:0000:0100:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [30]:
#training 


In [24]:

# check version number
import imblearn
print(imblearn.__version__)

0.10.1


In [28]:
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
under = RandomUnderSampler(sampling_strategy=0.5)


In [29]:
under = RandomUnderSampler(sampling_strategy=0.5)

In [30]:
#Split the data into train and test
df_train, df_val = train_test_split(df, test_size=0.25, shuffle=True)

print(len(df_train), len(df_val))

3329 1110


In [31]:
df_train.head()

,id,class,text
752,GPT4All,other,REFERENCE DOCUMENT AIR LIQUIDE FINANCIAL STATE...
1480,report_711.pdf.57,environmental,Diesel consumption kl Production kt Intensitie...
833,GPT4All,other,contract asset liability satisfied life contra...
292,GPT4All,other,Audit Committee Oversight financial reporting ...
638,GPT4All,environmental,Recycle Reusable Resources Program Our busines...


In [32]:
#balance
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# Separate features and target variable
X = df[['id', 'text']]
y = df['class']

# Create the RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

# Resample the data
X_resampled, y_resampled = rus.fit_resample(X, y)

# Create a new balanced DataFrame
df_balanced = pd.DataFrame(X_resampled, columns=['id', 'text'])
df_balanced['class'] = y_resampled

# Print the balanced DataFrame
print(df_balanced)

                      id                                               text  \
0            LLM created  Sustainable mining involves identifying way ba...   
1                GPT4All  Introduction Our Communities Our People Our En...   
2     report_1404.pdf.20  DIAMONDBACK ENERGY CSR ENVIRONMENT E N V I R O...   
3                GPT4All  Our environmental record Nevertheless believe ...   
4      report_567.pdf.17  Recycle Reusable Resources Program Our busines...   
...                  ...                                                ...   
2627             GPT4All  COMMUNITY ACCO Brands Charitable Giving Commun...   
2628   report_932.pdf.17  POLYMETAL EMPLOYEES Fatalities Severe injury M...   
2629             GPT4All  Everything today ensures safer productive tomo...   
2630             GPT4All  CASE STUDY area executive video Johan Meyer Ex...   
2631         LLM created  Embracing pluralism foster broader representat...   

              class  
0     environmental  
1     e

In [33]:
#an even set of classes for the training data
df_balanced['class'].value_counts()

environmental    658
governance       658
other            658
social           658
Name: class, dtype: int64

In [44]:
#df_balanced.to_csv('training_balanced_cleaned.csv')